In [4]:
import cv2 as cv
import numpy as np
import pandas as pd
import time
from datetime import datetime

In [15]:
# here we will be seeing how to create a motion detection and tracking system using
# OpenCV - Our target is to track a moving object - if motion is not happening, the 
# system wont be tracking it
## first thing first, lets read the video
cap = cv.VideoCapture("./vtest.avi")
# cap = cv.VideoCapture(0)

## to display the frames of the video
while cap.isOpened():
    _, frame = cap.read()
    cv.imshow("FRAME", frame)
    
    if cv.waitKey(50) == 27:
        break
cv.destroyAllWindows()
cap.release()

In [76]:
## lets get started:
## first we need a two frames outside of while loop which we can use inside the loop
# re-using the above code

cap = cv.VideoCapture("./vtest.avi")

## frame-1 & 2
_, frame1 = cap.read()
_, frame2 = cap.read()


## to display the frames of the video
while cap.isOpened():
    # we need to find the absolute difference between above frame
    abDiff = cv.absdiff(frame1, frame2)
    
    ## converting the abDiff to grayscale to find the contours
    gray = cv.cvtColor(abDiff, cv.COLOR_BGR2GRAY)
    
    ## we need to smoothen the image - using the gaussian blur
    blur = cv.GaussianBlur(gray, (5,5), 0)
    
    ## finding the threshold
    _, thres = cv.threshold(blur, 20, 255, cv.THRESH_BINARY)
    
    ## to find the bettet contours, lets dillate the threshold image
    dilate = cv.dilate(thres, None, iterations=3) ## if we wont specify the size of the kernel,
                                                   ## default value will be 3x3
    ## lets find the contours now
    contours, _ = cv.findContours(dilate, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    ## next step is to draw contours
    cv.drawContours(frame1, contours, -1, (0, 0, 255), 2)
    
    ## now to show the feed
    cv.imshow("VideoFeed", frame1)
    
    
    # now to find the frame 1 & 2
    frame1 = frame2
    _, frame2 = cap.read()
    
    
    if cv.waitKey(40) == 27:
        break
        
        
cv.destroyAllWindows()
cap.release()

In [75]:
## however, this is not the output we were looking for - we need to draw the rectangle
## instead of contours - lets see how to do that - reusing the code
## lets get started:

cap = cv.VideoCapture("./vtest.avi")

## frame-1 & 2
_, frame1 = cap.read()
_, frame2 = cap.read()


## to display the frames of the video
while cap.isOpened():
    # we need to find the absolute difference between above frame
    abDiff = cv.absdiff(frame1, frame2)
    
    ## converting the abDiff to grayscale to find the contours
    gray = cv.cvtColor(abDiff, cv.COLOR_BGR2GRAY)
    
    ## we need to smoothen the image - using the gaussian blur
    blur = cv.GaussianBlur(gray, (5,5), 0)
    
    ## finding the threshold
    _, thres = cv.threshold(blur, 20, 255, cv.THRESH_BINARY)
    
    ## to find the bettet contours, lets dillate the threshold image
    dilate = cv.dilate(thres, None, iterations=3) ## if we wont specify the size of the kernel,
                                                   ## default value will be 3x3
    ## lets find the contours now
    contours, _ = cv.findContours(dilate, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    ## next step is to draw contours
    #cv.drawContours(frame1, contours, -1, (0, 0, 255), 2) # we dont need to draw the 
                                                            # contours - need to draw
                                                             # rectangel
    
    ## to do that, we need to iterate over the contours list
    for contour in contours:
        # first we need to get the x,y coordinates - for that we can use method
         # cv.boundingRect()
        (x, y, w, h) = cv.boundingRect(contour)
        
        ## now, as we seen in the video that contours were also being drawn on the 
         # moving line - which we dont want that - we need to draw them on the persons
            # for that we can use the if conditon to skip the areas lesser than a value
        if cv.contourArea(contour) < 900:
            continue
        cv.rectangle(frame1, (x,y), (x+w, y+h), (0,255,0), 2)
        
        ## we can also put text in the video
        cv.putText(frame1, "Status: {}".format("Movement"), (10,20), cv.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)
    ## now to show the feed
    cv.imshow("VideoFeed", frame1)
    
    
    # now to find the frame 1 & 2
    frame1 = frame2
    _, frame2 = cap.read()
    
    
    if cv.waitKey(40) == 27:
        break
        
        
cv.destroyAllWindows()
cap.release()

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:650: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'cv::arithm_op'


In [73]:
## lets take it to a little bit advance level. We will also be storing the start time and end time of the motio
## This time we will be using webcam - i have already imported all the required package

## creating a variable to later used to compare and find the difference between
## two frames. As in motion detection, we continously finding the difference between
## two frames

## creating a static variable to later use it
static_back = None

## creating a list to later append the state of the motion - whenever the object will move
motion_list = [0, 0]

## time of the movement
time = []

## creating a dataframe with the start-time of the motion and end time of the motion
df = pd.DataFrame(columns = ['start', 'end'])

## lets capture the video
cap = cv.VideoCapture("./vtest.avi")



## lets use the loop to repeat the above process, with a small modifications
while True:
    
    
    ## reading a feed from camera
    _, frame = cap.read()
    
    # starting the motion variable 
    motion = 0
    
    ## lets convert it into grascale to process it
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    ## applying gaussian to find the change easily
    blur = cv.GaussianBlur(gray, (21,21), 0)
    
    ## using the static variable
    if static_back is None:
        static_back = gray
        continue
    
    ## taking absDifference value
    abDiff = cv.absdiff(static_back, blur)
    
    ## now converting it into a mask
    _, thres = cv.threshold(abDiff, 30, 255, cv.THRESH_BINARY)
    
    ## lets dillate the boundaries
    dilate = cv.dilate(thres, (3,3), iterations=2)
    
    ## now that we have our dilated mask, lets find the contours
    contours, _ = cv.findContours(dilate.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    ## now lets draw them, first find the motion which we will append in the motion list
    
    for contour in contours:
        if cv.contourArea(contour)<400:
            continue
 
        motion = 1
    
        ## lets get the x,y coordinates
        (x,y,w,h) = cv.boundingRect(contour)
        
        ## lets draw them
        cv.rectangle(frame, (x,y), (x+w, y+h), (255, 0, 0), 3)
    ## lets append the motion in the motion list
    motion_list.append(motion)
    
    ## now lets get the last two motions
    motion_list = motion_list[-2:]
    
    ## lets get the start and end time of the above motions
    if motion_list[-1] == 1 and motion_list[-2] == 0:
        time.append(datetime.now())
        
    if motion_list[-1] == 0 and motion_list[-2] == 1:
        time.append(datetime.now())
        
    ## now lets show the frames
    cv.imshow("FRAME", frame)
    
    
    if cv.waitKey(40) == 27:
        if motion == 1:
            time.append(datetime.now())
        break

for i in range(0, len(time), 2):
    df = df.append({'start':time[i], 'end':time[i+1]}, ignore_index=True)

df.to_csv("MovementTime.csv")

## releasing the sources
cap.release()
cv.destroyAllWindows()
        

C:\Users\muhammad.adil\AppData\Local\Temp\ipykernel_29816\144652161.py:94: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'start':time[i], 'end':time[i+1]}, ignore_index=True)


In [54]:
## the above code does not look so effective


[datetime.datetime(2022, 9, 20, 18, 55, 27, 250643),
 datetime.datetime(2022, 9, 20, 18, 55, 27, 441947),
 datetime.datetime(2022, 9, 20, 18, 55, 27, 505788),
 datetime.datetime(2022, 9, 20, 18, 55, 35, 265755),
 datetime.datetime(2022, 9, 20, 18, 55, 35, 317163),
 datetime.datetime(2022, 9, 20, 18, 55, 38, 81698)]

In [68]:

## lets take it to a little bit advance level. We will also be storing the start time and end time of the motio
## This time we will be using webcam - i have already imported all the required package

## creating a variable to later used to compare and find the difference between
## two frames. As in motion detection, we continously finding the difference between
## two frames

## creating a static variable to later use it


## creating a list to later append the state of the motion - whenever the object will move
motion_list = [0, 0]

## time of the movement
time = []

## creating a dataframe with the start-time of the motion and end time of the motion
df = pd.DataFrame(columns = ['start', 'end'])

## lets capture the video
cap = cv.VideoCapture(0)

_, frame1 = cap.read()
_, frame2 = cap.read()


## lets use the loop to repeat the above process, with a small modifications
while True:
    
    abDiff = cv.absdiff(frame1, frame2)
    ## reading a feed from camera
#     _, frame = cap.read()
    
    # starting the motion variable 
    motion = 0
    
    ## lets convert it into grascale to process it
    gray = cv.cvtColor(abDiff, cv.COLOR_BGR2GRAY)
    
    ## applying gaussian to find the change easily
    blur = cv.GaussianBlur(gray, (21,21), 0)
    
    ## using the static variable
#     if static_back is None:
#         static_back = gray
#         continue
    
    ## taking absDifference value
#     abDiff = cv.absdiff(frame2, blur)
    
    ## now converting it into a mask
    _, thres = cv.threshold(blur, 30, 255, cv.THRESH_BINARY)
    
    ## lets dillate the boundaries
    dilate = cv.dilate(thres, (3,3), iterations=2)
    
    ## now that we have our dilated mask, lets find the contours
    contours, _ = cv.findContours(dilate, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    ## now lets draw them, first find the motion which we will append in the motion list
    
    for contour in contours:
        if cv.contourArea(contour)<10000:
            continue
 
        motion = 1
    
        ## lets get the x,y coordinates
        (x,y,w,h) = cv.boundingRect(contour)
        
        ## lets draw them
        cv.rectangle(frame1, (x,y), (x+w, y+h), (255, 0, 0), 3)
    ## lets append the motion in the motion list
    motion_list.append(motion)
    
    ## now lets get the last two motions
    motion_list = motion_list[-2:]
    
    ## lets get the start and end time of the above motions
    if motion_list[-1] == 1 and motion_list[-2] == 0:
        time.append(datetime.now())
        
    if motion_list[-1] == 0 and motion_list[-2] == 1:
        time.append(datetime.now())
        
    ## now lets show the frames
    cv.imshow("FRAME", frame1)
    
    frame1 = frame2
    _, frame2 = cap.read()
    
    if cv.waitKey(40) == 27:
        if motion == 1:
            time.append(datetime.now())
        break

for i in range(0, len(time), 2):
    df = df.append({'start':time[i], 'end':time[i+1]}, ignore_index=True)

df.to_csv("MovementTime.csv")

## releasing the sources
cap.release()
cv.destroyAllWindows()
        

C:\Users\muhammad.adil\AppData\Local\Temp\ipykernel_29816\894545240.py:98: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'start':time[i], 'end':time[i+1]}, ignore_index=True)
C:\Users\muhammad.adil\AppData\Local\Temp\ipykernel_29816\894545240.py:98: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'start':time[i], 'end':time[i+1]}, ignore_index=True)
C:\Users\muhammad.adil\AppData\Local\Temp\ipykernel_29816\894545240.py:98: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'start':time[i], 'end':time[i+1]}, ignore_index=True)
C:\Users\muhammad.adil\AppData\Local\Temp\ipykernel_29816\894545240.py:98: FutureWarning: The frame.append method is deprecated and will be removed from pandas 

In [61]:
frame2.shape

(480, 640, 3)

In [64]:
blur.shape

(480, 640)